In [52]:
import json
import os
import pandas as pd

jsonFOlderPath='D:\\PhD\\Dataset\\CourtOrdersJson\\DistrictandSessionCourtPune\\CriBailAppln'

df = pd.DataFrame()

for subdir, dirs, files in os.walk(jsonFOlderPath):
    for filename in files:
        filepath = subdir + os.sep + filename

        if filepath.endswith(".json"):            
            f = open(filepath,"r", encoding="utf8")
            data = json.load(f) 
            df=df.append(data, ignore_index = True)

In [55]:
print(df.shape)
print(df.columns)

(470, 12)
Index(['actualOrders', 'caseNumber', 'caseType', 'caseYear',
       'courtEstablishment', 'decisionReasons', 'decisions', 'id',
       'judgements', 'parties', 'rawTextOrder', 'title'],
      dtype='object')


In [62]:
print(df['decisions'][0])

[{'decisionSequence': 1, 'decisionPoint': 'Application is Accepted or Allowed?', 'decisionFinding': 'NO'}]
